---
authors:
  - name: Mathis Derenne
    affiliation: M2 MIASHS - Université de Lyon
    github: mathisdrn
  - name: Djida Boukari
    affiliation: M2 MIASHS - Université de Lyon
  - name: Ines Nakhli
    affiliation: M2 MIASHS - Université de Lyon
date: 2026-02-01
---

# Modèle Random Survival Forest

## Imports

In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs

## Chargement des données

In [ ]:
from sksurv.util import Surv

df_train = pl.read_parquet("../../data/df_study_L18_w6_train.parquet")
df_test = pl.read_parquet("../../data/df_study_L18_w6_test.parquet")

X_train = df_train.drop(["event", "time"]).to_pandas()
X_test = df_test.drop(["event", "time"]).to_pandas()

y_train = Surv.from_dataframe("event", "time", df_train.to_pandas())
y_test = Surv.from_dataframe("event", "time", df_test.to_pandas())

## Entraînement du modèle

In [11]:
from sksurv.ensemble import RandomSurvivalForest

rsf = RandomSurvivalForest(
    n_estimators=400,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features="sqrt",
    n_jobs=-1,
)

rsf.fit(X_train, y_train)

## Évaluation du modèle final

In [13]:
from utils import evaluate_survival_model

risk_rsf = rsf.predict(X_test)
surv_rsf = rsf.predict_survival_function(X_test)
evaluate_survival_model(df_train, df_test, risk_rsf, surv_rsf)

C-index,Mean td-AUC,IBS,Brier (t=1210.0)
0.707,0.769,0.173,0.186
